In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import random
import re
import folium

### Nom de la commune associé au code postal

In [2]:
# dataset 
path = 'C:/Users/taminot/Documents/conf/TIDE/data/'
path = ''
geo = pd.read_csv(f"{path}correspondance-code-insee-code-postal.csv",
                 sep=";",
                 )
geo_prefecture = geo[(geo['Statut']=='Préfecture') | (geo['Statut']=='Préfecture de région') & (geo['Code Département']!='97')]
geo_prefecture.loc[:,'Commune'] = geo_prefecture.loc[:,'Commune'].apply(lambda x: x if not ('ARRONDISSEMENT' in x) else re.search('\\w+', x).group())

historique = pd.read_csv(f"{path}historique.csv", index_col=0)

geo_prefecture.head()

/home/tanguy/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,Code INSEE,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geogla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
338,77288,77000,MELUN,SEINE-ET-MARNE,ILE-DE-FRANCE,Préfecture,63.0,800.0,39.4,"48.5444723553, 2.65795821917","{""type"": ""Polygon"", ""coordinates"": [[[2.653274...",36389,288,99,2,77,11
496,15014,15000,AURILLAC,CANTAL,AUVERGNE,Préfecture,668.0,2916.0,28.2,"44.9245233686, 2.44162453828","{""type"": ""Polygon"", ""coordinates"": [[[2.483154...",4256,14,98,1,15,83
526,2A004,20000,AJACCIO,CORSE-DU-SUD,CORSE,Préfecture de région,136.0,8314.0,64.3,"41.9347926638, 8.70132275974","{""type"": ""Polygon"", ""coordinates"": [[[8.636808...",5174,4,98,1,2A,94
540,60057,60000,BEAUVAIS,OISE,PICARDIE,Préfecture,91.0,3309.0,54.5,"49.4365523321, 2.08616123661","{""type"": ""Polygon"", ""coordinates"": [[[2.125963...",28861,57,99,1,60,22
652,44109,44000/44100/44200/44300,NANTES,LOIRE-ATLANTIQUE,PAYS DE LA LOIRE,Préfecture de région,18.0,6577.0,282.0,"47.2316356767, -1.54831008605","{""type"": ""Polygon"", ""coordinates"": [[[-1.51819...",419,109,99,2,44,52


### Carte

In [3]:
import folium

In [4]:
fond = r'http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'
carte = folium.Map(location=[46.5, 2.3], zoom_start=6, tiles=fond, attr='© OpenStreetMap © CartoDB')
#carte.create_map(path='C:/Users/taminot/Documents/Projets persos/Départements/image.html')
carte.save(f"{path}image.html")

jsonData = f"{path}contour-des-departements.geojson"
folium.Choropleth(jsonData, fill_color='yellow').add_to(carte)

# JEU CONNAITRE LES PREFECTURE DES CODES POSTAUX

In [5]:
def jeu_prefecture(graph=True):
    nb_tour = int(input('Nombre de tour ? '))
    erreur = 0
    code_list = geo_prefecture['Code Département'].unique().tolist()
    for i in range(nb_tour):
        Code = random.choice(code_list)
        code_list.remove(Code)
        Commune = geo_prefecture.loc[geo_prefecture['Code Département']==Code, 'Commune'].tolist()[0]
        Commune_joueur = input(f"Quelle est la préfécture associé au numéro de département {Code} : ")
        if Commune_joueur.lower() != Commune.lower():
            print(f"ERROR : La préfécture est {Commune.lower()}")
            erreur +=1
            historique.loc[Code,'Erreur'] = historique.loc[Code,'Erreur'] + 1
            if graph:
                '''
                Init carte
                '''
                fond = r'http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'
                carte = folium.Map(location=[46.5, 2.3], zoom_start=6, tiles=fond, attr='© OpenStreetMap © CartoDB')
                #carte.create_map(path='C:/Users/taminot/Documents/Projets persos/Départements/image.html')
                carte.save(f"{path}image.html")

                jsonData = f"{path}contour-des-departements.geojson"
                folium.Choropleth(jsonData, fill_color='bisque').add_to(carte)

                chaine = geo_prefecture.loc[geo_prefecture['Code Département']==Code, 'geo_point_2d'].to_list()[0]
                ind = chaine.index(',')
                x = chaine[0:ind]
                y = chaine[(ind+2):]
                folium.Marker([x, y], popup=f"{Code} : {Commune}").add_to(carte)
                display(carte)
        else:
            historique.loc[Code,'Correct'] = historique.loc[Code,'Correct'] + 1
    historique.to_csv(f"{path}historique.csv")
    print(f"Votre score est de {nb_tour-erreur}/{nb_tour}")

In [ ]:
jeu_prefecture(graph=False)

Nombre de tour ? 20
Quelle est la préfécture associé au numéro de département 04 : 
ERROR : La préfécture est digne-les-bains
Quelle est la préfécture associé au numéro de département 67 : Strasbourg
Quelle est la préfécture associé au numéro de département 09 : 
ERROR : La préfécture est foix
Quelle est la préfécture associé au numéro de département 87 : Limoge
ERROR : La préfécture est limoges
Quelle est la préfécture associé au numéro de département 89 : Auxerre
Quelle est la préfécture associé au numéro de département 43 : 
ERROR : La préfécture est le puy-en-velay
Quelle est la préfécture associé au numéro de département 18 : Bourges
Quelle est la préfécture associé au numéro de département 63 : Clermont-ferrand
Quelle est la préfécture associé au numéro de département 21 : dijon
Quelle est la préfécture associé au numéro de département 66 : perpignan
Quelle est la préfécture associé au numéro de département 39 : lagon-le-saunier
ERROR : La préfécture est lons-le-saunier
Quelle es

### Resultats

In [ ]:
result = (historique['Correct']>historique['Erreur']).sum()/historique.shape[0]
"{:.2%}".format(result)

#### Avec le plus d'erreurs

In [ ]:
historique.sort_values('Erreur', ascending=False)

#### Ceux qui sont pas loin d'etre connus

In [ ]:
filtre = (historique['Correct'].apply(lambda x: True if x in [1, 2] else False)) & (historique['Erreur']>0)
historique[filtre].sort_values(by=['Correct', 'Erreur'] ,ascending = [False, True]) 

### APPRENDRE LES PREFECTURES

In [ ]:
def apprendre_prefecture(graph = True):
    nb_tour = int(input('Nombre de tour ? '))
    erreur = 0
    filtre = (historique['Correct']<3) #2*historique['Erreur'])
    geo_prefecture_apprendre = historique[filtre]
    code_list = geo_prefecture_apprendre.index.unique().tolist()
    for i in range(nb_tour):
        Code = random.choice(code_list)
        code_list.remove(Code)
        Commune = geo_prefecture_apprendre.loc[Code, 'Commune']
        Commune_joueur = input(f"Quelle est la préfécture associé au numéro de département {Code} : ")
        if Commune_joueur.lower() != Commune.lower():
            print(f"ERROR : La préfécture est {Commune.lower()}")
            erreur +=1
            historique.loc[Code,'Erreur'] = historique.loc[Code,'Erreur'] + 1
            if graph:
                '''
                Init carte
                '''
                fond = r'http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'
                carte = folium.Map(location=[46.5, 2.3], zoom_start=6, tiles=fond, attr='© OpenStreetMap © CartoDB')
                #carte.create_map(path='C:/Users/taminot/Documents/Projets persos/Départements/image.html')
                carte.save(f"{path}image.html")

                jsonData = f"{path}contour-des-departements.geojson"
                folium.Choropleth(jsonData, fill_color='bisque').add_to(carte)

                chaine = geo_prefecture.loc[geo_prefecture['Code Département']==Code, 'geo_point_2d'].to_list()[0]
                ind = chaine.index(',')
                x = chaine[0:ind]
                y = chaine[(ind+2):]
                folium.Marker([x, y], popup=f"{Code} : {Commune}").add_to(carte)
                display(carte)
        else:
            historique.loc[Code,'Correct'] = historique.loc[Code,'Correct'] + 1
    historique.to_csv(f"{path}historique.csv")
    print(f"Votre score est de {nb_tour-erreur}/{nb_tour}")

In [ ]:
apprendre_prefecture()

### JEUX DES VILLES (PREFECTURE) A PLACER : 

In [ ]:
def jeu_ville_placer():
    list_commune = geo_prefecture['Commune'].unique().tolist()
    Commune = random.choice(list_commune)
    Code = geo_prefecture.loc[geo_prefecture.Commune==Commune, 'Code Département'].tolist()[0]
    '''
    Init carte
    '''
    fond = r'http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'
    carte = folium.Map(location=[46.5, 2.3], zoom_start=6, tiles=fond, attr='© OpenStreetMap © CartoDB')
    #carte.create_map(path='C:/Users/taminot/Documents/Projets persos/Départements/image.html')
    carte.save('C:/Users/taminot/Documents/Projets persos/Départements/image.html')

    jsonData = r'C:/Users/taminot/Documents/Projets persos/Départements/contour-des-departements.geojson'
    folium.Choropleth(jsonData, fill_color='bisque').add_to(carte)

    chaine = geo_prefecture.loc[geo_prefecture['Code Département']==Code, 'geo_point_2d'].to_list()[0]
    ind = chaine.index(',')
    x = chaine[0:ind]
    y = chaine[(ind+2):]
    folium.Marker([x, y], popup=f"{Code} : {Commune}").add_to(carte)
    display(carte)

In [ ]:
jeu_ville_placer()

In [ ]:
def prefecture(code):
    ville = geo_prefecture.loc[geo_prefecture['Code Département']==code,'Commune'].to_list()[0]
    print(f"{code} : {ville}")
def code_prefecture(ville):
    ville = ville.upper()
    code = geo_prefecture.loc[geo_prefecture['Commune']==ville,'Code Département'].to_list()[0]
    print(f"{ville} : {code}")

In [ ]:
prefecture('54')

In [ ]:
code_prefecture('Besancon')

In [ ]:
geo_prefecture[geo_prefecture['Commune']=='MARSEILLE']

In [ ]:
geo_prefecture.sort_values('Population', ascending = False)

In [ ]:
historique

In [ ]:
(historique['Correct']>historique['Erreur']).sum()/historique.shape[0]